In [1]:
import pandas as pd
import requests
import country_converter as coco
from bs4 import BeautifulSoup
import folium
from sqlalchemy import create_engine
from os import getenv
from datetime import date

In [2]:
engine = create_engine(getenv('POSTGRES_CONN'))

# Достану из википедии и сохраню в БД таблицы кодов стран и штатов США.
И сделаю эти коды первичными ключами.

### Таблица всех стран и кодов Alpha 3

In [3]:
url_countries = 'https://en.wikipedia.org/wiki/ISO_3166-1'

df_countries = pd.read_html(url_countries)[1]
df_countries.rename(columns={'English short name (using title case)': 'country', 
                             'Alpha-3 code': 'country_code', 
                             'Independent': 'independent'}, inplace=True)
df_countries = df_countries.iloc[:, [2, 0, 5]]
for country_code, row in df_countries.iterrows():
    if country_code != 'VGB' and country_code != 'VIR':
        row['country'] = row['country'].strip('"')
        row['country'] = row['country'].split(' (')[0]
        row['country'] = row['country'].split('[')[0]
        row['country'] = row['country'].split(',')[0]

df_countries.head(2)

,country_code,country,independent
0,AFG,Afghanistan,Yes
1,ALA,Åland Islands,No


In [4]:
df_countries.to_sql('countries', engine, index=False)
with engine.connect() as con:
    con.execute("""ALTER TABLE countries ADD CONSTRAINT PK_country_code PRIMARY KEY (country_code);""")

### Таблица всех штатов США в формате ISO

In [5]:
url_states = 'https://en.wikipedia.org/wiki/ISO_3166-2:US'
df_states = pd.read_html(url_states)[0]
df_states.columns = ['state_code', 'state_name', 'category']
df_states.head(2)

,state_code,state_name,category
0,US-AL,Alabama,State
1,US-AK,Alaska,State


In [6]:
df_states.to_sql('states', engine, index=False)
with engine.connect() as con:
    con.execute("""ALTER TABLE states ADD CONSTRAINT PK_state_code PRIMARY KEY (state_code);""")

# Numbeo. Анализ и скрапинг городов и их индексов.

За основу беру таблицу Numbeo "Current Cost of Living Index", в которой на данный момент 506 городов и, пока, не понятно сколько стран и по сколько городов в этих странах.

In [7]:
url = 'https://www.numbeo.com/cost-of-living/rankings_current.jsp'
df = pd.read_html(url)[1]
df.head(2)

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,"Hamilton, Bermuda",163.04,105.83,135.98,167.42,156.39,88.45
1,NaN,"Zurich, Switzerland",120.44,61.92,92.76,122.01,114.80,117.16


Добавляю другие интексы Numbeo относящиеся к городам. В разных таблицах индексы дублируются, но охватывают разное количество городов. Выбираю те, в которых больше городов.

In [8]:
indexes_dict = {
    'https://www.numbeo.com/quality-of-life/rankings_current.jsp': 
        ['City',
         'Quality of Life Index',
         'Property Price to Income Ratio',
         'Traffic Commute Time Index',
         'Climate Index'],
    'https://www.numbeo.com/crime/rankings_current.jsp': ['City', 'Safety Index'],
    'https://www.numbeo.com/health-care/rankings_current.jsp': ['City', 'Health Care Index'],
    'https://www.numbeo.com/pollution/rankings_current.jsp': ['City', 'Pollution Index']
}

In [9]:
for key, value in indexes_dict.items():
    df_temp = pd.read_html(key)[1][value]
    df = df.merge(df_temp, how='left', on='City')
    
df.drop(['Rank', 'Restaurant Price Index'], axis=1, inplace=True)
df.columns = list(map(lambda x: x.removesuffix(' Index').lower().replace(' ', '_'), df.columns))
df.rename(columns={'city': 'city_country'}, inplace=True)
#df.head(3)

Добавляю в таблицу ссылки на страницы данных городов со строгим указанием валюты USD

In [10]:
response = requests.get(url).text
soup = BeautifulSoup(response, 'lxml')
links = soup.find_all('td', class_='cityOrCountryInIndicesTable')

href_list = []
for i in links:
    href_list.append([i.find('a').text, i.find('a').get('href') + '?displayCurrency=USD'])
    
city_link = pd.DataFrame(href_list, columns=['city_country', 'link'])

df = df.merge(city_link, how='left', on='city_country')

### Разделить Город - Штат - Страна

In [11]:
split_table1 = df['city_country'].str.rsplit(', ', n=1, expand=True)
split_table1.columns=['city_state', 'country']

split_table2 = split_table1['city_state'].str.split(', ', n=1, expand=True)
split_table2.columns=['city', 'state_code']

df = pd.concat([split_table2, split_table1['country'], df],axis=1)

df['city'] = df.apply(lambda row: row.city.split(' (')[0], axis = 1) #убираю дубли городов в скобках

#df.head()

### Добавляю alpha_3 код к странам

Сначала удалю Косово, тк у него нет alpha3-кода.

In [12]:
df = df.drop(df.loc[df['country'] == 'Kosovo (Disputed Territory)'].index)

In [13]:
new_df = pd.DataFrame({'country': df['country'].unique()})
new_df['country_code'] = new_df.apply(lambda row: coco.convert(names=row.country, to='ISO3') , axis = 1)
df = df.merge(new_df, how='left', on='country')

#df['country_code'] = df.apply(lambda row: coco.convert(names=row.country, to='ISO3') , axis = 1) - можно в одну строчку, но дольше в 4 раза
df.head(3)

,city,state_code,country,city_country,cost_of_living,rent,cost_of_living_plus_rent,groceries,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution,link,country_code
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",163.04,105.83,135.98,167.42,88.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Hamil...,BMU
1,Zurich,None,Switzerland,"Zurich, Switzerland",120.44,61.92,92.76,122.01,117.16,195.15,10.06,32.62,81.48,82.64,74.84,17.58,https://www.numbeo.com/cost-of-living/in/Zuric...,CHE
2,Basel,None,Switzerland,"Basel, Switzerland",119.65,45.21,84.44,126.05,108.89,195.80,8.23,20.79,82.82,79.89,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Basel...,CHE


### Добавляю код к штатам USA
Есть три значения штатов, не относящихся к штатам США.
Всем этим значениям присвою знанчение None.
А к штатам США добавлю "US- " - приведу к ISO стандарту обозначения штатов США.

In [14]:
df[df['state_code'].notnull() & (df['country'] != "United States")]

,city,state_code,country,city_country,cost_of_living,rent,cost_of_living_plus_rent,groceries,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution,link,country_code
47,Nanaimo,BC,Canada,"Nanaimo, BC, Canada",78.64,37.59,59.22,81.02,83.95,NaN,NaN,NaN,NaN,50.77,68.65,NaN,https://www.numbeo.com/cost-of-living/in/Nanai...,CAN
72,St. John's,Newfoundland and Labrador,Canada,"St. John's, Newfoundland and Labrador, Canada",75.71,23.13,50.84,75.34,143.97,NaN,NaN,NaN,NaN,53.06,NaN,NaN,https://www.numbeo.com/cost-of-living/in/St-Jo...,CAN
449,Batumi,Ajara,Georgia,"Batumi, Ajara, Georgia",32.15,12.93,23.06,31.83,26.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Batum...,GEO


In [15]:
indexes = df[df['state_code'].notnull() & (df['country'] != "United States")]['state_code'].index
df.loc[indexes, 'state_code'] = None

df['state_code'] = df.apply(lambda row: f"US-{row['state_code']}" if row['state_code'] != None else None, axis = 1)

# Карта
В индексе Numbeo 505 городов, но не понятно, какие страны, много их или мало. Сколько городов в этих странах и в каких пропорциях. Суммарные цифры мне мало что скажут, гораздо понятнее будет отобразить их на карте.

In [16]:
map_df = df.groupby('country_code')['city'].nunique().reset_index().sort_values('city', ascending=False)

m = folium.Map(location=[37.87820990704326, 6.555063556986549], zoom_start=1.5)
folium.Choropleth(
    geo_data='scripts/data/world.geojson',
    name="choropleth",
    data=map_df,
    columns=['country_code', 'city'],
    key_on="feature.properties.ISO_A3", 
    fill_color="YlGn",
    nan_fill_color='pink',
    fill_opacity=0.8,
    bins=7,
    reset=True,
    highlight=True,
    legend_name='Count of cities'
).add_to(m)

In [17]:
#m - show interactive map by folium

![title](./data/map.png)

На github интерактивная карта не отображается, поэтому вместо вывода карты, вставил картинку.

Теперь видно какие страны есть в индексе, а каких нет, и сколько городов в каждой стране. Меня всё устраивает, вот только мне не нужно в Индии так много городов. Удалю лишние (оставлю 2 города).

In [18]:
india = df.loc[(df['country'] == 'India') & 
              (df['city'] != 'Delhi') & 
              (df['city'] !='Mumbai')]

df = df.drop(india.index)

Подготовлю индексы (Primary Key) для будущей таблицы в postgres, чтобы сразу можно было загрузить её в БД.

In [19]:
df.reset_index(drop=True, inplace=True)
df.insert(0, 'city_id', df.index +1)
df.head(3)

,city_id,city,state_code,country,city_country,cost_of_living,rent,cost_of_living_plus_rent,groceries,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution,link,country_code
0,1,Hamilton,None,Bermuda,"Hamilton, Bermuda",163.04,105.83,135.98,167.42,88.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Hamil...,BMU
1,2,Zurich,None,Switzerland,"Zurich, Switzerland",120.44,61.92,92.76,122.01,117.16,195.15,10.06,32.62,81.48,82.64,74.84,17.58,https://www.numbeo.com/cost-of-living/in/Zuric...,CHE
2,3,Basel,None,Switzerland,"Basel, Switzerland",119.65,45.21,84.44,126.05,108.89,195.80,8.23,20.79,82.82,79.89,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Basel...,CHE


Добавляю в таблицу 'state_name' из БД. И сохраняю с нужными колонками и Primary Keys в индексе для скрапинга в файл.

In [20]:
states_tbl = pd.read_sql('states', engine).loc[:, ['state_code', 'state_name']]
df = df.merge(states_tbl, on='state_code', how='left')
df_to_file = df.iloc[:, [0, 1, 2, -1, 3, -2, 4, -3]].copy()
df_to_file.set_index('city_id', inplace=True)
df_to_file.to_pickle("./scripts/data/numbeo_links.pkl")

Формирую и загружаю в БД таблицу городов с информацией о том кем добавлены данные в таблицу и назначаю ключи

In [21]:
df_cities = df.iloc[:, [0, 1, 2, -2]].copy()
df_cities.to_sql('cities', engine, if_exists='append', index=False)
with engine.connect() as con:
    con.execute("""ALTER TABLE cities ADD CONSTRAINT PK_city_id PRIMARY KEY (city_id);
                ALTER TABLE cities ADD CONSTRAINT FK_country_code FOREIGN KEY (country_code) REFERENCES public.countries (country_code);
                ALTER TABLE cities ADD CONSTRAINT FK_state_code FOREIGN KEY (state_code) REFERENCES public.states (state_code);""")

# Формирую таблицу Индексов намбео по городам + загрузка в БД

In [22]:
df_city_index = df.iloc[:, [0] + list(range(5,17))].copy()
# добавляю столбцы когда и кем добавлены данные в таблицу
df_city_index.loc[:, ('sys_updated_date', 'sys_updated_by')] = (date.today(), getenv('NB_USER'))
df_city_index.head(2)

,city_id,cost_of_living,rent,cost_of_living_plus_rent,groceries,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution,sys_updated_date,sys_updated_by
0,1,163.04,105.83,135.98,167.42,88.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-17,analyst_k2
1,2,120.44,61.92,92.76,122.01,117.16,195.15,10.06,32.62,81.48,82.64,74.84,17.58,2022-06-17,analyst_k2


In [23]:
df_city_index.to_sql('numbeo_indexes_by_city', engine, index=False)
with engine.connect() as con:
    con.execute("""ALTER TABLE numbeo_indexes_by_city ADD CONSTRAINT FK_numbeo_indexes_by_city FOREIGN KEY (city_id) REFERENCES public.cities (city_id);""")

# Формирую таблицу Индексов намбео по странам + загрузка в БД

#### From Cost of Living Index by Country 2022

In [24]:
df_col_index = pd.read_html('https://www.numbeo.com/cost-of-living/rankings_by_country.jsp')[1]
df_col_index.drop('Rank', axis=1, inplace=True)
df_col_index.columns = list(map(lambda x: x.removesuffix(' Index').lower().replace(' ', '_'), df_col_index.columns))

#### From Quality of Life Index by Country

In [25]:
df_qol_index = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp')[1]
df_qol_index.drop(['Rank', 'Cost of Living Index', 'Safety Index'], axis=1, inplace=True)
df_qol_index.columns = list(map(lambda x: x.removesuffix(' Index').lower().replace(' ', '_'), df_qol_index.columns))

#### From Average Monthly Net Salary by Country

In [26]:
df_salary_index = pd.read_html('https://www.numbeo.com/cost-of-living/country_price_rankings?itemId=105')[1]
df_salary_index.drop([0, 2], axis=1, inplace=True)
df_salary_index[3] = df_salary_index[3].apply(lambda row: float(row.rstrip('\xa0$').replace(',', '')))
df_salary_index.rename(columns={1: 'country', 3: 'avg_salary(usd)'}, inplace=True)

#### From Crime Index by Country

In [27]:
df_safety_index = pd.read_html('https://www.numbeo.com/crime/rankings_by_country.jsp')[1]
df_safety_index = df_safety_index[['Country', 'Safety Index']]
df_safety_index.columns=['country', 'safety']

#### Merge 4 Country Tables

In [28]:
df_country_index = df_col_index.merge(df_qol_index, how='left', on='country')
df_country_index = df_country_index.merge(df_salary_index, how='left', on='country')
df_country_index = df_country_index.merge(df_safety_index, how='left', on='country')
# добавляю столбцы когда и кем добавлены данные в таблицу
df_country_index.loc[:, ('sys_updated_date', 'sys_updated_by')] = (date.today(), getenv('NB_USER'))

Удаляю Косово

In [29]:
df_country_index = df_country_index.drop(df_country_index.loc[df_country_index['country'] == 'Kosovo (Disputed Territory)'].index)

In [30]:
df_country_index['country_code'] = df_country_index.apply(lambda row: coco.convert(names=row.country, to='ISO3') , axis = 1)
df_country_index = df_country_index.reindex(columns=['country_code'] + list(df_country_index.columns[1:-1]))
df_country_index.head(2)

,country_code,cost_of_living,rent,cost_of_living_plus_rent,groceries,restaurant_price,local_purchasing_power,quality_of_life,purchasing_power,health_care,property_price_to_income_ratio,traffic_commute_time,pollution,climate,avg_salary(usd),safety,sys_updated_date,sys_updated_by
0,BMU,146.04,98.58,123.80,148.66,159.17,81.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-17,analyst_k2
1,CHE,123.35,53.54,90.62,128.13,122.09,118.44,195.27,118.44,74.85,8.29,28.5,19.59,80.21,5892.78,78.32,2022-06-17,analyst_k2


In [31]:
df_country_index.to_sql('numbeo_indexes_by_country', engine, if_exists='append', index=False)
with engine.connect() as con:
    con.execute("""ALTER TABLE numbeo_indexes_by_country ADD CONSTRAINT FK_country_code FOREIGN KEY (country_code) REFERENCES public.countries (country_code);""")